<a href="https://colab.research.google.com/github/TanjilaNur/Age-Transformation-Prediction/blob/main/Age_Transformation_Prediction_from_an_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/TanjilaNur/Age-Transformation-Prediction.git
%cd Age-Transformation-Prediction/
!pip3 install -r requirements.txt

Cloning into 'Age-Transformation-Prediction'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 46 (delta 11), reused 41 (delta 9), pack-reused 0
Unpacking objects: 100% (46/46), done.
/content/Age-Transformation-Prediction
     |████████████████████████████████| 686kB 3.0MB/s 
     |████████████████████████████████| 245kB 17.0MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp37-none-any.whl size=655251 sha256=b8a2b8017da81e408ec2872add3e65e13d6706a42835b62c396f7c5438900eeb
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=b1e22ab95c6dc5ed322b92e03b05c9718890f30000086aa6b10211a3d4ab85f7
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e6

In [3]:
!python download_models.py

100% 213M/213M [00:06<00:00, 35.2MB/s]
Extracting males model zip file
Done!
100% 213M/213M [00:04<00:00, 45.9MB/s]
Extracting females model zip file
Done!
100% 99.7M/99.7M [00:05<00:00, 18.2MB/s]
Done!
100% 178M/178M [00:04<00:00, 37.1MB/s]
Done!
100% 464M/464M [00:08<00:00, 57.9MB/s]
Done!


In [ ]:
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

In [ ]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)

In [ ]:
opt.name = 'females_model' # change to 'females_model' if you're trying the code on a female image
model = create_model(opt)
model.eval()

In [ ]:
# upload your image (the code supports only a single image at a time)
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  img_path = filename
  print('User uploaded file "{name}"'.format(name=filename))

In [ ]:
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

In [ ]:
 use_webm = False
# For some unknown reason the mp4 video is not displayed on chrome
# If you have chrome, uncomment the following lines to convert the 
# result to webm for display purposes

 !pip3 install webm
 webm_out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.webm')
 !webm -i $out_path $webm_out_path
 use_webm = True

from IPython.display import HTML
from base64 import b64encode
video_path = webm_out_path if use_webm else out_path
video_type = "video/webm" if use_webm else "video/mp4"
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width={0} controls>
      <source src="{1}" type="{2}">
</video>
""".format(opt.fineSize, data_url, video_type))

In [ ]:
files.download(out_path)